In [ ]:
# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/blip2_qformer.py#L101
# Blip2Qformer的forward方法
query_output = self.Qformer.bert(
    query_embeds=query_tokens,
    encoder_hidden_states=image_embeds,
    encoder_attention_mask=image_atts,
    use_cache=True,
    return_dict=True,
)

这里计算的是queries的self attention，图片的self attention，以及两者的cross attention。

self.Qformer.bert是BertModel类。

In [ ]:
# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/Qformer.py#L804
# BertModel类的forward方法
def forward(
    self,
    input_ids=None,
    attention_mask=None,
    position_ids=None,
    head_mask=None,
    query_embeds=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None,
    is_decoder=False,
):

In [ ]:
在ITC任务的图片特征计算中，`BertModel.forward()`的张量参数有且只有：

- query_embeds，queries向量
- encoder_hidden_states，图片向量
- encoder_attention_mask，图片attention mask

这里有个细节，图片向量不计算self attention，直接作为encoder_hidden_states，和queries计算cross attention。

In [ ]:
# BertModel类的forward方法
past_key_values_length = (
    past_key_values[0][0].shape[2] - self.config.query_length
    if past_key_values is not None
    else 0
)

past_key_values_length=0

In [ ]:
query_length = query_embeds.shape[1] if query_embeds is not None else 0

query_length = 32

In [ ]:
# BertModel类的forward方法
embedding_output = self.embeddings(
    input_ids=input_ids,
    position_ids=position_ids,
    query_embeds=query_embeds,
    past_key_values_length=past_key_values_length,
)

这块代码只有当提供input_ids时才实际运行，否则embedding_output就是query_embeds。

In [ ]:
input_shape = embedding_output.size()[:-1]
batch_size, seq_length = input_shape

seq_length = query_length

In [ ]:
if attention_mask is None:
    attention_mask = torch.ones(
        ((batch_size, seq_length + past_key_values_length)), device=device
)

attention_mask是queries的mask。

In [ ]:
if is_decoder:
   # ...
else:
    extended_attention_mask = self.get_extended_attention_mask(
        attention_mask, input_shape, device, is_decoder
    )

is_decoder=False

In [ ]:
# BertModel类的get_extended_attention_mask方法
if attention_mask.dim() == 3: 
    # ...
elif attention_mask.dim() == 2:
    if is_decoder:
        # ...
    else:
        extended_attention_mask = attention_mask[:, None, None, :]
extended_attention_mask = extended_attention_mask.to(
    dtype=self.dtype
)  # fp16 compatibility
extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
return extended_attention_mask

attention_mask[:, None, None, :]将attention_mask扩展为[batch_size, num_heads=1, seq_length=1, seq_length]。
num_heads和seq_length=1在应用的时候，会被广播到相应的size。
第一个 seq_length：表示查询序列的长度，即我们正在处理的输入序列的每个位置。第二个 seq_length：表示key序列的长度，通常等于查询序列的长度。

attention_mask的1是有效位置，0是无效位置。(1.0 - extended_attention_mask) * -10000.0操作后，有效位置变成0，无效位置-10000. 这样在softmax之后，无效位置接近于0.

In [ ]:
# BertModel类的forward方法
if encoder_hidden_states is not None:
    # ...
else:
    encoder_extended_attention_mask = None
# Prepare head mask if needed
# 1.0 in head_mask indicate we keep the head
# attention_probs has shape bsz x n_heads x N x N
# input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
# and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

输入head_mask=None，那么所有head都不被屏蔽。

In [ ]:
# BertModel类的forward方法
encoder_outputs = self.encoder(
    embedding_output,
    attention_mask=extended_attention_mask,
    head_mask=head_mask,
    encoder_hidden_states=encoder_hidden_states,
    encoder_attention_mask=encoder_extended_attention_mask,
    past_key_values=past_key_values,
    use_cache=use_cache,
    output_attentions=output_attentions,
    output_hidden_states=output_hidden_states,
    return_dict=return_dict,
    query_length=query_length,
)

# BertEncoder的forward方法
def forward(
    self,
    hidden_states,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True,
    query_length=0,
):

In [ ]:
hidden_states=query_embeds
attention_mask=extended_attention_mask
head_mask=head_mask
encoder_hidden_states=image_embeds
encoder_attention_mask=image_atts
use_cache=True
output_attentions=None
output_hidden_states=None
return_dict=True
query_length=query_length

In [ ]:
# BertEncoder的forward方法
all_hidden_states = () if output_hidden_states else None # None
all_self_attentions = () if output_attentions else None # None
all_cross_attentions = (
    () if output_attentions and self.config.add_cross_attention else None
) # None

next_decoder_cache = () if use_cache else None # ()

In [ ]:
# BertEncoder的forward方法
for i in range(self.config.num_hidden_layers):
    layer_module = self.layer[i]
    if output_hidden_states:
        # ...

    layer_head_mask = head_mask[i] if head_mask is not None else None
    past_key_value = past_key_values[i] if past_key_values is not None else None # None

    if getattr(self.config, "gradient_checkpointing", False) and self.training:
        # 略
    else:
        layer_outputs = layer_module(
            hidden_states,
            attention_mask,
            layer_head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
            query_length,
        )
    hidden_states = layer_outputs[0]
    if use_cache:
        next_decoder_cache += (layer_outputs[-1],)
    if output_attentions:
        # ...
        
if output_hidden_states:
    # ...

if not return_dict:
    # ...
return BaseModelOutputWithPastAndCrossAttentions(
    last_hidden_state=hidden_states,
    past_key_values=next_decoder_cache,
    hidden_states=all_hidden_states,
    attentions=all_self_attentions,
    cross_attentions=all_cross_attentions,
)

In [ ]:
# BertLayer的forward方法
def forward(
    self,
    hidden_states,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_value=None,
    output_attentions=False,
    query_length=0,
):

In [ ]:
hidden_states=query_embeds
attention_mask=extended_attention_mask
head_mask=head_mask
encoder_hidden_states=image_embeds
encoder_attention_mask=image_atts
past_key_value=None
output_attentions=None
output_hidden_states=None
query_length=query_length

In [ ]:
# BertLayer的forward方法
# decoder uni-directional self-attention cached key/values tuple is at positions 1,2
self_attn_past_key_value = (
    past_key_value[:2] if past_key_value is not None else None
) # None
self_attention_outputs = self.attention(
    hidden_states,
    attention_mask,
    head_mask,
    output_attentions=output_attentions,
    past_key_value=self_attn_past_key_value,
)
attention_output = self_attention_outputs[0]
outputs = self_attention_outputs[1:-1]

present_key_value = self_attention_outputs[-1]

计算queries的self attention。
self_attention_outputs是三元组，元素分别是：
- 元素0：attention_output这个是自注意力层的主要输出，是经过注意力机制计算后的隐状态。它的形状通常为 (batch_size, sequence_length, hidden_size)。
- 元素1：attention_weights（如果 output_attentions=True）：这个是注意力权重（attention weights），表示计算注意力得分时的权重矩阵。它的形状通常为 (batch_size, num_heads, sequence_length, sequence_length)。
- 元素2：present_key_value：这个是用来保存注意力计算中键和值、的缓存，用于加速后续的推理或者训练。它的形状通常是 ((batch_size, num_heads, sequence_length, head_dim), (batch_size, num_heads, sequence_length, head_dim))，分别对应键和值。

In [ ]:
# BertLayer的forward方法
if query_length > 0:
    query_attention_output = attention_output[:, :query_length, :]

    if self.has_cross_attention: # True
        assert (
            encoder_hidden_states is not None
        ), "encoder_hidden_states must be given for cross-attention layers"
        cross_attention_outputs = self.crossattention(
            query_attention_output,
            attention_mask,
            head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            output_attentions=output_attentions,
        )
        query_attention_output = cross_attention_outputs[0]
        outputs = (
            outputs + cross_attention_outputs[1:-1]
        )  # add cross attentions if we output attention weights

    layer_output = apply_chunking_to_forward(
        self.feed_forward_chunk_query,
        self.chunk_size_feed_forward,
        self.seq_len_dim,
        query_attention_output,
    )
    
    if attention_output.shape[1] > query_length: # False
        # ...
outputs = (layer_output,) + outputs

outputs = outputs + (present_key_value,)

return outputs

query_attention_output = attention_output[:, :query_length, :]在这里是多余的，因为attention_output本来就是queries的self attention。

self.crossattention计算queries和image_embeds的cross attention。

apply_chunking_to_forward是transformers自带的函数。处理长序列输入时，内存占用可能非常高。通过分块处理，可以将输入序列分成若干个小块，每次只处理一个小块，从而减少单次计算所需的内存量。chunk_size_feed_forward定义在 https://huggingface.co/docs/transformers/main_classes/configuration , 如果bert-base-uncased没有配置，则默认为0. The chunk size of all feed forward layers in the residual attention blocks. A chunk size of 0 means that the feed forward layer is not chunked. A chunk size of n means that the feed forward layer processes n < sequence_length embeddings at a time. 

outputs是3元组，元素分别是：
- 注意力机制计算后的隐藏状态
- attention_weights
- present_key_value

In [ ]:
# BertEncoder的forward方法
for i in range(self.config.num_hidden_layers):
    # ...
    else:
        layer_outputs = layer_module(
            hidden_states
            # ...
        )
    hidden_states = layer_outputs[0]
    if use_cache:
        next_decoder_cache += (layer_outputs[-1],)
    if output_attentions:
        # ...
        
return BaseModelOutputWithPastAndCrossAttentions(
    last_hidden_state=hidden_states,
    past_key_values=next_decoder_cache,
    hidden_states=all_hidden_states,
    attentions=all_self_attentions,
    cross_attentions=all_cross_attentions,
)

逐层计算self attention和cross attention，返回最后一层的隐状态。

In [ ]:
# BertModel类的forward方法
encoder_outputs = self.encoder(
    # ...
    )
sequence_output = encoder_outputs[0]
pooled_output = (
    self.pooler(sequence_output) if self.pooler is not None else None
) # None

if not return_dict:
    # ...

return BaseModelOutputWithPoolingAndCrossAttentions(
    last_hidden_state=sequence_output,
    pooler_output=pooled_output,
    past_key_values=encoder_outputs.past_key_values,
    hidden_states=encoder_outputs.hidden_states,
    attentions=encoder_outputs.attentions,
    cross_attentions=encoder_outputs.cross_attentions,
)

In [ ]:
sequence_output是逐层计算self attention和cross attention，返回最后一层的隐状态的结果。

没用pooling，最终返回的还是sequence_output。

In [ ]:
# Blip2Qformer的forward方法
query_output = self.Qformer.bert(
    query_embeds=query_tokens,
    encoder_hidden_states=image_embeds,
    encoder_attention_mask=image_atts,
    use_cache=True,
    return_dict=True,
)

# ...

text_output = self.Qformer.bert(
    text_tokens.input_ids,
    attention_mask=text_tokens.attention_mask,
    return_dict=True,
)

计算完图片特征后，再计算文本特征。

In [ ]:
# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/Qformer.py#L804
# BertModel类的forward方法
def forward(
    self,
    input_ids=None,
    attention_mask=None,
    position_ids=None,
    head_mask=None,
    query_embeds=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None,
    is_decoder=False,
):

In [ ]:
在ITC任务的文本特征计算中
- input_ids=text_tokens.input_ids
- attention_mask=text_tokens.attention_mask
- return_dict=True
- 其他默认

In [ ]:
# BertModel类的forward方法
# ...
query_length = query_embeds.shape[1] if query_embeds is not None else 0 #0
embedding_output = self.embeddings(
    input_ids=input_ids,
    position_ids=position_ids,
    query_embeds=query_embeds,
    past_key_values_length=past_key_values_length,
)

# ...
if is_decoder:
    # ...
else:
    extended_attention_mask = self.get_extended_attention_mask(
        attention_mask, input_shape, device, is_decoder
    )

# ...

encoder_outputs = self.encoder(
    embedding_output,
    attention_mask=extended_attention_mask,
    head_mask=head_mask,
    encoder_hidden_states=encoder_hidden_states,
    encoder_attention_mask=encoder_extended_attention_mask,
    past_key_values=past_key_values,
    use_cache=use_cache,
    output_attentions=output_attentions,
    output_hidden_states=output_hidden_states,
    return_dict=return_dict,
    query_length=query_length,
)

query_length=0
extended_attention_mask的计算方式和图片特征中的计算同理。



In [ ]:
# BertEncoder类的forward方法
def forward(
    self,
    hidden_states,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True,
    query_length=0,
):



In [ ]:
hidden_states=embedding_output
attention_mask=extended_attention_mask
head_mask=head_mask
encoder_hidden_states=None
encoder_attention_mask=None
use_cache=True
output_attentions=None
output_hidden_states=None
return_dict=True
query_length=0

In [ ]:
# BertEncoder类的forward方法
# ...
for i in range(self.config.num_hidden_layers):
    layer_module = self.layer[i]
    # ...
    if getattr(self.config, "gradient_checkpointing", False) and self.training:
        # ...
    else:
        layer_outputs = layer_module(
            hidden_states,
            attention_mask,
            layer_head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
            query_length,
        )
#...

到这里，queries的self attention, 和图片向量的cross attention, 文本的self attention，就都计算完成了。